In [1]:
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("firebase-credentials.json")
firebase_admin.initialize_app(cred)

In [5]:
from firebase_admin import firestore

db = firestore.client()

In [9]:
for doc in db.collection('urls').stream():
    print(f'{doc.id} => {doc.to_dict()}')

test => {'date': DatetimeWithNanoseconds(2023, 2, 18, 17, 0, 0, 896000, tzinfo=datetime.timezone.utc), 'username': 'arvind6902@gmail.com', 'url': 'https://www.google.com/'}


In [27]:
username = 'arvind6902@gmail.com'
old_url_docs = db.collection('urls').where('username', '==', username).stream()
for doc in old_url_docs:
    print(doc.to_dict())

{'title': 'Apple', 'timestamp': '11:00', 'username': 'arvind6902@gmail.com', 'url': 'https://www.apple.com/'}
{'title': 'Google Maps', 'timestamp': '10:00', 'url': 'https://www.google.com/maps', 'username': 'arvind6902@gmail.com'}


In [32]:
clusters = db.collection('clusters').where('username', '==', username).stream()
for cluster in clusters:
    print(cluster.reference.update({
        'abc': 'def'
    }))

update_time {
  seconds: 1676750069
  nanos: 980595000
}

update_time {
  seconds: 1676750070
  nanos: 58867000
}

